# Supervised Machine Learning - Data Classification Demo
## Part 3 - MODEL TRAINING

In this notebook, we load the processed dataset file and use it to train several classification models.

> **INPUT:** the ready dataset csv file as cleaned and processed in the previous phases.<br>
> **OUTPUT:** a comparison of the prediction accuracy and performance of multiple machine learning classification algorithms.  

***

### 1. INITIALIZATION

In [11]:
#importing required libraries and modules
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split

### 2. LOADING PROCESSED DATASET

#### Reading dataset file into pandas DataFrame

In [7]:
#initialize required variables to read the cleaned data file
data_file_location = "..\\data\\processed\\"
data_file_name = "conn.log.labeled_processed"
data_file_ext = ".csv"


#read the dataset
data_df = pd.read_csv(data_file_location + data_file_name + data_file_ext, index_col=0)

#### Exploring dataset summary and statistics

In [8]:
#check dataset shape
data_df.shape

(23145, 33)

In [9]:
#check dataset head
data_df.head()

,id.orig_h,id.orig_p,id.resp_h,id.resp_p,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,...,conn_state_S3,conn_state_SF,history_C,history_D,history_Dd,history_Other,history_S,history_ShAdDaf,history_ShAdDaft,history_ShAdfDr
0,3.232236e+09,41040.0,3.119782e+09,80.0,3.139211,0.0,0.0,0.0,3.0,180.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,3.232236e+09,41040.0,3.119782e+09,80.0,3.152487,0.0,0.0,0.0,1.0,60.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,3.232236e+09,41040.0,3.119782e+09,80.0,3.152487,0.0,0.0,0.0,1.0,60.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,3.232236e+09,41040.0,3.119782e+09,80.0,1.477656,149.0,128252.0,2896.0,94.0,5525.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,3.232236e+09,41042.0,3.119782e+09,80.0,3.147116,0.0,0.0,0.0,3.0,180.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
